In [2]:
#Step 1, load all the preprocessed text files and indicate the number of unique characters per file 
import os
import shutil
import pandas as pd

# Define the folder path
folder_path = r'C:\Users\prathyusha\Documents\Annual Reports\text files\preprocessed'

# List all files in the folder
files = os.listdir(folder_path)

# Create an empty list to store the data
data = []

# Iterate through the files, rename them, and capture the content
for file_name in files:
    # Get the full file path
    file_path = os.path.join(folder_path, file_name)
    
    # Extract the part of the filename after the last backslash
    new_name = file_name.rsplit('\\', 1)[-1]
    
    # Construct the new file path with the extracted name
    new_path = os.path.join(folder_path, new_name)
    
    # Rename the file
    shutil.move(file_path, new_path)
    
    # Read the contents of the file
    with open(new_path, 'r', encoding='utf-8') as file:
        content = file.read()
    
    # Calculate the character count
    char_count = len(content)
    
    # Append the data to the list, including the content
    data.append({'Filename': new_name, 'Content': content, 'Character Count': char_count})

# Create a DataFrame from the list of data
df = pd.DataFrame(data)

# Print the DataFrame
print(df)
print(df.iloc[1])


#Step 2, within dataframe df, create a vaiable for observations with a value less than 50 for Character Count. Then create a variable to indicate the number of unique characters in Character Count
# Filter rows where 'Character Count' is greater than or equal to 50

# Apply the function to each row and create the new variable
df['FewTotalCharacters'] = df['Character Count'].apply(lambda x: 1 if x < 50 else 0)

# Use value_counts to count the occurrences of each unique value in the "FewTotalCharacters" column
value_counts_table = df['FewTotalCharacters'].value_counts().reset_index()
value_counts_table.columns = ['Value', 'Count']
print(value_counts_table)

# Define a function to calculate the number of unique words
def calculate_unique_words(text):
    words = text.split()
    unique_words = set(words)
    return len(unique_words)

# Apply the function to each row and create the new variable "Unique Word Count"
df['Unique Word Count'] = df['Content'].apply(calculate_unique_words)

# Create a new variable "LessThan50" with a value of 1 if "Unique Word Count" is less than 50, else 0
df['FewUniqueWords'] = df['Unique Word Count'].apply(lambda x: 1 if x < 50 else 0)

# Use value_counts to count the occurrences of each unique value in the "FewUniqueCharacters" column
value_counts_table = df['FewUniqueWords'].value_counts().reset_index()
value_counts_table.columns = ['Value', 'Count']
print(value_counts_table)

##Problematic reports denoted by 1
df['ProblematicReport'] = df['FewTotalCharacters'] + df['FewUniqueWords']
# Clip the values in the "ProblematicReport" column to a maximum of 1
df['ProblematicReport'] = df['ProblematicReport'].clip(upper=1)

value_counts_table = df['ProblematicReport'].value_counts().reset_index()
value_counts_table.columns = ['Value', 'Count']
print(value_counts_table)


###get rid of white space
import re


# Define a function to remove punctuation   ###PLEASE NOTE ON 10.13.2023 I removed this part of the code. The decimal places shoud not be problematic and it may be valuable to keep them in. 
#def remove_punctuation(text):
#    # Use regular expressions to remove punctuation
#    text = re.sub(r'[^\w\s]', '', text)
#    return text

# Apply the remove_punctuation function to the "Content" column
#df['Content'] = df['Content'].apply(remove_punctuation)

# Define a function to clean whitespace
def clean_whitespace(text):
    # Replace multiple consecutive spaces or whitespace characters with one space
    cleaned_text = re.sub(r'\s+', ' ', text)
    return cleaned_text

# Apply the custom function to each row in the DataFrame
df['Content'] = df['Content'].apply(clean_whitespace)

# Print the DataFrame with cleaned whitespace
print(df['Content'])

# Print the DataFrame
print(df)
print(df.iloc[1])
print(df["Content"].iloc[2])

####recognize how many times the following phrases appears in each row of dataframe df for variable Content: “shareholder letter”, “letter for shareholder

# Define the phrases to search for
phrases_to_search = ["shareholder letter", "letter for shareholder", "dear shareholder", "to our shareholder", "fellow shareholder",
                     "stockholder letter", "letter for stockholder", "dear stockholder", "to our stockholder", "fellow stockholder", 
                     "shareowner letter", "letter for shareowner", "dear shareowner", "to our shareowner", "fellow shareowner", 
                     "stakeholder letter", "letter for stakeholder", "dear stakeholder", "to our stakeholder", "fellow stakeholder",
                     "letter to shareholder", "letter to stockholder", "letter to shareowner", "letter to stakeholder",
                     "letter to the shareholder", "letter to the stockholder", "letter to the shareowner", "letter to the stakeholder",
                     "investor letter", "letter for investor", "dear investor", "to our investor", "fellow investor",
                     "message from the chairman", "message from our chairman", "message from the ceo", "message from the chief executive officer",
                     "letter for valued shareholder", "dear valued shareholder", "to our valued shareholder", "fellow valued shareholder",
                     "letter for valued stockholder", "dear valued stockholder", "to our valued stockholder", "fellow valued stockholder", 
                     "letter for valued shareowner", "dear valued shareowner", "to our valued shareowner", "fellow valued shareowner", 
                     "letter for valued stakeholder", "dear valued stakeholder", "to our valued stakeholder", "fellow valued stakeholder",
                     "letter to valued stockholder", "letter to valued shareowner", "letter to valued stakeholder",
                     "letter to the valued stockholder", "letter to the valued shareowner", "letter to the valued stakeholder",]

# Create a custom function to count occurrences of phrases in a given text
def count_phrases(text):
    text = text.lower()  # Convert text to lowercase for case-insensitive search
    text = text.replace("letters", "letter")  # Replace "letters" with "letter"
    text = text.replace("shareholders", "shareholder")  # Replace "shareholders" with "shareholder"
    text = text.replace("stockholders", "stockholder")  
    text = text.replace("shareowners", "shareowner") 
    text = text.replace("investors", "investor") 
    text = text.replace("valuable", "valued") 
    text = text.replace("cherished", "valued") 
    text = text.replace("owner", "shareowner") 
    text = text.replace("l e t t e r", "letter") 
    text = text.replace("s h a r e h o l d e r", "shareholder")  # Replace "shareholders" with "shareholder"
    text = text.replace("s t o c k o l d e r", "stockholder")  
    text = text.replace("s h a r e o w n e r", "shareowner") 
    text = text.replace("i n v e s t o r", "investor") 
    text = text.replace("v a l u a b l e", "valued") 
    text = text.replace("c h e r i s h e d", "valued") 
    text = text.replace("v a l u e d", "valued") 
    text = text.replace("o w n e r", "shareowner") 
    text = text.replace("c e o", "ceo") 
    text = text.replace("c h i e f", "chief") 
    text = text.replace("e x e c u t i v e", "executive") 
    text = text.replace("o f f i c e r", "officer") 
    text = text.replace("c h a i r m a n", "chairman") 
    
    count = 0
    for phrase in phrases_to_search:
        count += text.count(phrase.lower())
    return count

# Apply the custom function to each row in the DataFrame
df['Phrase_Count'] = df['Content'].apply(count_phrases)

value_counts_table = df['Phrase_Count'].value_counts().reset_index()
value_counts_table.columns = ['Value', 'Count']
print(value_counts_table)




# Define the phrases to search for
additional_phrases_to_search = ["a message from"]

# Create a custom function to count occurrences of phrases in a given text
def count_phrases(text):
    text = text.lower()  # Convert text to lowercase for case-insensitive search
    
    count = 0
    for phrase in additional_phrases_to_search:
        count += text.count(phrase.lower())
    return count

# Apply the custom function to each row in the DataFrame
df['Additional_Phrase_Count'] = df['Content'].apply(count_phrases)

value_counts_table = df['Additional_Phrase_Count'].value_counts().reset_index()
value_counts_table.columns = ['Value', 'Count']
print(value_counts_table)



###Get rid of what can be problematic parts of the report
# Define the phrases to search for
problematic_phrases_to_search = ["united states securities and exchange commission"]#note that this often references key phrases related to the shareholder letter but is not the shareholder. we know it almost always appears after a shareholder letter so we will use that info later to remove this

# Create a custom function to count occurrences of phrases in a given text
def count_phrases(text):
    text = text.lower()  # Convert text to lowercase for case-insensitive search
    
    count = 0
    for phrase in problematic_phrases_to_search:
        count += text.count(phrase.lower())
    return count

# Apply the custom function to each row in the DataFrame
df['Problematic_Phrase_Count'] = df['Content'].apply(count_phrases)

value_counts_table = df['Problematic_Phrase_Count'].value_counts().reset_index()
value_counts_table.columns = ['Value', 'Count']
print(value_counts_table)



###

import os
import pandas as pd

# Create a new variable "Name" containing character values from "Filename"
df['Name'] = df['Filename'].str.extract('([A-Za-z]+)', expand=False)

# Create a new variable "Year" containing numerical values from "Filename"
df['Year'] = df['Filename'].str.extract('(\d+)', expand=False)

# Convert the "Year" column to numeric (if it's not already)
df['Year'] = pd.to_numeric(df['Year'], errors='coerce')

# Print the DataFrame with the new variables
print(df)




#df_revised = df.drop(columns=['Content'])


# Define the folder path to save the Excel file
#folder_path = r'C:\Users\hartmannn\Desktop\Extraction\preprocessed'

# Specify the Excel file name for the revised DataFrame
#excel_file_name = 'revised_dataframe.xlsx'


# Create the full file path
#excel_file_path = os.path.join(folder_path, excel_file_name)

# Save the revised DataFrame as an Excel file
#df_revised.to_excel(excel_file_path, index=False)


# Print a message to confirm the save
#print(f"Revised DataFrame saved as {excel_file_path}")




###Step 3. Recognize how many times the following words appear in the document. Trying to pick up on the end of the shareholder letter. Running this as is so the number of phrases can be expanded to names if warranted.
df['Presence_Shareholder_Letter'] = df['Phrase_Count'] + df['Additional_Phrase_Count'] 
df['Presence_Shareholder_Letter'] = df['Presence_Shareholder_Letter'].clip(upper=1)


# Define the phrases to search for
end_phrases_to_search = ["ceo", "chief executive officer"]

# Create a custom function to count occurrences of phrases in a given text
def count_phrases(text):
    text = text.lower()  # Convert text to lowercase for case-insensitive search
    
    count = 0
    for phrase in end_phrases_to_search:
        count += text.count(phrase.lower())
    return count

# Apply the custom function to each row in the DataFrame
df['End_Phrase_Count'] = df['Content'].apply(count_phrases)

value_counts_table = df['End_Phrase_Count'].value_counts().reset_index()
value_counts_table.columns = ['Value', 'Count']
print(value_counts_table)


###Step 4. Create a new content variable to hold shareholder letters for which we will manipulate. 
# we will drop to the right for where the united states securities and exchange commission phrase is introduced 
# We will then drop contents to the left of the first time a key phrase noting the start of a shareholder letter. 
# Then will then  drop to the right of a key phrase noting the end of a shareholder letter.
#  This will not give us perfect results. We would need to fine tune the results more to get exactly what we need 

df['Rev_Content'] = df['Content']


#deleting aspects of the document unlikely to be important, afterthe shareholder letter. Note we only keep to the left of our problematic phrase (i.e., we delete to the right of the problem phrase)
def trim_after_problematic_phrase(text, problematic_phrases_to_search):
    # Convert the text to lowercase for a case-insensitive search
    text_lower = text.lower()
    
    # Look for the first occurrence of any of the problematic phrases in the text
    first_occurrences = [text_lower.find(phrase) for phrase in problematic_phrases_to_search if text_lower.find(phrase) != -1]

    # If any of the problematic phrases were found
    if first_occurrences:
        # Get the earliest occurrence position
        earliest_occurrence = min(first_occurrences)
        # Return the content up to that point
        return text[:earliest_occurrence]

    # If no problematic phrase found, return the original text
    return text

# Apply the function
df['Rev_Content'] = df['Rev_Content'].apply(lambda content: trim_after_problematic_phrase(content, problematic_phrases_to_search))




#deleting aspects of the document unlikely to be important, before the shareholder letter
def extract_content_after_phrase(text):
    # Convert text to lowercase for case-insensitive search
    text_lower = text.lower()
    
    # Find the first occurrence of any of the phrases
    first_occurrences = [text_lower.find(phrase) for phrase in phrases_to_search if text_lower.find(phrase) != -1]

    # If any of the phrases were found in the text
    if first_occurrences:
        # Get the earliest occurrence position
        earliest_occurrence = min(first_occurrences)
        # Return the content from that point onward
        return text[earliest_occurrence:]
    # If none of the phrases were found, return the original text
    return text

# Apply the function to the 'Rev_Content' column
df['Rev_Content'] = df['Rev_Content'].apply(extract_content_after_phrase)



###STEP 4b. Drop after the end of the shareholder letter. This will be an imperfect measure 

def trim_content_after_end_phrase(text, end_phrases_to_search):
    # Convert the text to lowercase for case-insensitive search
    text_lower = text.lower()
    
    # Find the position after the first # words
    words = text.split()
    position_after_499_words = len(' '.join(words[:499])) + (1 if len(words) > 499 else 0)  # Add 1 to account for the space after the 149th word

    # Look for the first occurrence of any of the end phrases after that position
    first_occurrences = [text_lower.find(phrase, position_after_499_words) for phrase in end_phrases_to_search if text_lower.find(phrase, position_after_499_words) != -1]

    # If any of the end phrases were found after the first # words
    if first_occurrences:
        # Get the earliest occurrence position
        earliest_occurrence = min(first_occurrences)
        #  Return the content up to and including that point
        return text[:earliest_occurrence + len(end_phrases_to_search[first_occurrences.index(earliest_occurrence)])]

    # If conditions not met, return the original text
    return text

df['Rev_Content'] = df.apply(lambda row: trim_content_after_end_phrase(row['Rev_Content'], end_phrases_to_search) if row['Phrase_Count'] >= 1 or row['Additional_Phrase_Count'] >= 1 else row['Rev_Content'], axis=1)


###Write over existing columns
# Define the phrases to search for
phrases_to_search = ["shareholder letter", "letter for shareholder", "dear shareholder", "to our shareholder", "fellow shareholder",
                     "stockholder letter", "letter for stockholder", "dear stockholder", "to our stockholder", "fellow stockholder", 
                     "shareowner letter", "letter for shareowner", "dear shareowner", "to our shareowner", "fellow shareowner", 
                     "stakeholder letter", "letter for stakeholder", "dear stakeholder", "to our stakeholder", "fellow stakeholder",
                     "letter to shareholder", "letter to stockholder", "letter to shareowner", "letter to stakeholder",
                     "letter to the shareholder", "letter to the stockholder", "letter to the shareowner", "letter to the stakeholder",
                     "investor letter", "letter for investor", "dear investor", "to our investor", "fellow investor",
                     "message from the chairman", "message from our chairman", "message from the ceo", "message from the chief executive officer",
                     "letter for valued shareholder", "dear valued shareholder", "to our valued shareholder", "fellow valued shareholder",
                     "letter for valued stockholder", "dear valued stockholder", "to our valued stockholder", "fellow valued stockholder", 
                     "letter for valued shareowner", "dear valued shareowner", "to our valued shareowner", "fellow valued shareowner", 
                     "letter for valued stakeholder", "dear valued stakeholder", "to our valued stakeholder", "fellow valued stakeholder",
                     "letter to valued stockholder", "letter to valued shareowner", "letter to valued stakeholder",
                     "letter to the valued stockholder", "letter to the valued shareowner", "letter to the valued stakeholder",]

# Create a custom function to count occurrences of phrases in a given text
def count_phrases(text):
    text = text.lower()  # Convert text to lowercase for case-insensitive search
    text = text.replace("letters", "letter")  # Replace "letters" with "letter"
    text = text.replace("shareholders", "shareholder")  # Replace "shareholders" with "shareholder"
    text = text.replace("stockholders", "stockholder")  
    text = text.replace("shareowners", "shareowner") 
    text = text.replace("investors", "investor") 
    text = text.replace("valuable", "valued") 
    text = text.replace("cherished", "valued") 
    text = text.replace("owner", "shareowner") 
    text = text.replace("l e t t e r", "letter") 
    text = text.replace("s h a r e h o l d e r", "shareholder")  # Replace "shareholders" with "shareholder"
    text = text.replace("s t o c k o l d e r", "stockholder")  
    text = text.replace("s h a r e o w n e r", "shareowner") 
    text = text.replace("i n v e s t o r", "investor") 
    text = text.replace("v a l u a b l e", "valued") 
    text = text.replace("c h e r i s h e d", "valued") 
    text = text.replace("v a l u e d", "valued") 
    text = text.replace("o w n e r", "shareowner") 
    text = text.replace("c e o", "ceo") 
    text = text.replace("c h i e f", "chief") 
    text = text.replace("e x e c u t i v e", "executive") 
    text = text.replace("o f f i c e r", "officer") 
    text = text.replace("c h a i r m a n", "chairman") 
    
    count = 0
    for phrase in phrases_to_search:
        count += text.count(phrase.lower())
    return count

# Apply the custom function to each row in the DataFrame
df['A_Phrase_Count'] = df['Rev_Content'].apply(count_phrases)




# Define the phrases to search for
additional_phrases_to_search = ["a message from"]

# Create a custom function to count occurrences of phrases in a given text
def count_phrases(text):
    text = text.lower()  # Convert text to lowercase for case-insensitive search
    
    count = 0
    for phrase in additional_phrases_to_search:
        count += text.count(phrase.lower())
    return count

# Apply the custom function to each row in the DataFrame
df['A_Additional_Phrase_Count'] = df['Rev_Content'].apply(count_phrases)


df['A_Presence_Shareholder_Letter'] = df['A_Phrase_Count'] + df['A_Additional_Phrase_Count'] 
df['A_Presence_Shareholder_Letter'] = df['A_Presence_Shareholder_Letter'].clip(upper=1)

# Define the folder path to save the Excel file
folder_path = r'C:\Users\prathyusha\Documents\Annual Reports\text files\preprocessed'

# Specify the Excel file name for the revised DataFrame
excel_file_name = 'revised_dataframe.xlsx'

# Create the full file path
excel_file_path = os.path.join(folder_path, excel_file_name)

# Save the revised DataFrame as an Excel file
df.to_excel(excel_file_path, index=False)

# Print a message to confirm the save
print(f"Revised DataFrame saved as {excel_file_path}")






                                   Filename  \
0          2006 BERKLEY (W R) CORP_proc.txt   
1        2006 EVEREST RE GROUP LTD_proc.txt   
2    2006 MOLSON COORS BEVERAGE CO_proc.txt   
3                 2006 NETFLIX INC_proc.txt   
4                 2006 NVIDIA CORP_proc.txt   
..                                      ...   
356                2022 RESMED INC_proc.txt   
357       2022 ROCKWELL AUTOMATION_proc.txt   
358     2022 ROYAL CARIBBEAN GROUP_proc.txt   
359          2022 SCHLUMBERGER LTD_proc.txt   
360     2022 SCHWAB (CHARLES) CORP_proc.txt   

                                               Content  Character Count  
0    w. r. berkley corporation\n annual report\nlon...           252461  
1    everest re group ltd.\n annual report\ncorpora...           482865  
2    molson coors brewing company   annual report\n...           551263  
3     \nnetflix inc.   winchester circle  los gatos...           270774  
4    form ka\nnvidia corp  nvda\nfiled november   p...           